In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import re
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout


from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
#import stemmer as hindi_stemmer

In [2]:
%cd /content/drive/MyDrive/HASOC2021-EnglishHindiMarathi/trainingSet

/content/drive/MyDrive/HASOC2021-EnglishHindiMarathi/trainingSet


In [15]:
english_stopwords = stopwords.words("english")
with open('final_stopwords.txt', encoding = 'utf-8') as f:
    hindi_stopwords = f.readlines()
    for i in range(len(hindi_stopwords)):
        hindi_stopwords[i] = re.sub('\n','',hindi_stopwords[i])
stopwords = english_stopwords + hindi_stopwords
english_stemmer = SnowballStemmer("english")

In [4]:
df=pd.read_excel("en_Hasoc2021_train.xlsx")

In [6]:
df.head()

,Unnamed: 0,_id,text,task_1,task_2
0,4986,60c5d6bf5659ea5e55defa2c,@wealth if you made it through this &amp;&amp;...,HOF,PRFN
1,3394,60c5d6bf5659ea5e55def461,Technically that's still turning back the cloc...,HOF,OFFN
2,1310,60c5d6bf5659ea5e55defaad,@VMBJP @BJP4Bengal @BJP4India @narendramodi @J...,NOT,NONE
3,3390,60c5d6bf5659ea5e55def419,@krtoprak_yigit Soldier of Japan Who has dick ...,HOF,OFFN
4,4626,60c5d6bf5659ea5e55def7fa,@blueheartedly You'd be better off asking who ...,HOF,OFFN


## **Text preprocessing**

In [7]:
tweets = df.text
y = df.task_1

In [9]:
regex_for_english_hindi_emojis="[^a-zA-Z#\U0001F300-\U0001F5FF'|'\U0001F600-\U0001F64F'|'\U0001F680-\U0001F6FF'|'\u2600-\u26FF\u2700-\u27BF\u0900-\u097F]"
def clean_tweet(tweet):
    tweet = re.sub(r"@[A-Za-z0-9]+",' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet)
    tweet = re.sub(regex_for_english_hindi_emojis,' ', tweet)
    tweet = re.sub("RT ", " ", tweet)
    tweet = re.sub("\n", " ", tweet)
    tweet = re.sub(r" +", " ", tweet)
    tokens = []
    for token in tweet.split():
        if token not in stopwords:
            token = english_stemmer.stem(token)
            tokens.append(token)
    return " ".join(tokens)

In [16]:
cleaned_tweets = [clean_tweet(tweet) for tweet in tweets]

In [18]:
vectorizer = TfidfVectorizer(min_df = 5)
X = vectorizer.fit_transform(cleaned_tweets)
X = X.todense()

## **Training and testing**

In [19]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
y_pred = classifier.predict(X_val)

In [22]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.80      0.92      0.86       529
         NOT       0.73      0.50      0.59       240

    accuracy                           0.79       769
   macro avg       0.77      0.71      0.72       769
weighted avg       0.78      0.79      0.77       769



In [23]:
le = LabelEncoder() #label encoding labels for training Dense Neural Network
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

In [43]:
model = Sequential(
    [
        Dense(32, activation="relu"),
        Dropout(0.8),
        Dense(16, activation="relu"),
        Dropout(0.6),
        Dense(8, activation="sigmoid"),
        Dropout(0.4),
        Dense(1, activation="sigmoid"),
    ]
)
model.compile('adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [44]:
model.fit(X_train, y_train, epochs = 30, batch_size = 32)

Epoch 1/30
97/97 [==============================] - 1s 3ms/step - loss: 0.6631 - accuracy: 0.6184
Epoch 2/30
97/97 [==============================] - 0s 3ms/step - loss: 0.6721 - accuracy: 0.6041
Epoch 3/30
97/97 [==============================] - 0s 3ms/step - loss: 0.6549 - accuracy: 0.6272
Epoch 4/30
97/97 [==============================] - 0s 2ms/step - loss: 0.6499 - accuracy: 0.6314
Epoch 5/30
97/97 [==============================] - 0s 2ms/step - loss: 0.6316 - accuracy: 0.6558
Epoch 6/30
97/97 [==============================] - 0s 3ms/step - loss: 0.6056 - accuracy: 0.6812
Epoch 7/30
97/97 [==============================] - 0s 2ms/step - loss: 0.5883 - accuracy: 0.7066
Epoch 8/30
97/97 [==============================] - 0s 2ms/step - loss: 0.5623 - accuracy: 0.7196
Epoch 9/30
97/97 [==============================] - 0s 2ms/step - loss: 0.5380 - accuracy: 0.7466
Epoch 10/30
97/97 [==============================] - 0s 2ms/step - loss: 0.5177 - accuracy: 0.7664
Epoch 11/30
97/97 [

In [45]:
y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype('int64')
y_pred = y_pred.reshape(len(y_pred)) 

In [46]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.81      0.81       529
           1       0.59      0.62      0.60       240

    accuracy                           0.75       769
   macro avg       0.71      0.71      0.71       769
weighted avg       0.75      0.75      0.75       769

